<a href="https://colab.research.google.com/github/davidalejandromiranda/laboratorios-fisica/blob/master/blackbody.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"> Análisis de espectros de radiancia</h1>
<div align="right">Daniel A. Triana, Ph.D(c)<br>2020</div>

Con Spectraplot se puede descargar archivos con extensión ".csv" en esta herramienta se pueden analizar esos datos para obtener algunos parámetros de los espectros.

[Video explicativo](https://youtu.be/xXOhJ07xpFg)

##Importar librerias

En esta celda se importarán las librerías para: 
  * Cargar los archivos a Colab.
  * Graficar las curvas.
  * Importar archivos de extensión .csv y manejo de dataframe
  * Manejo de datos numéricos.
  * Optimización de datos.

In [ ]:
#instalando libreria Bokeh
!pip install bokeh
from google.colab import files
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from __future__ import print_function
import pandas as pd
import numpy as np
from scipy.optimize import basinhopping
# Configuración para mostrar gráficos en linea.
output_notebook()

## Celda para cargar archivos desde la computadora

In [ ]:
data = files.upload()

## Celda de extracción de datos del dataframe

In [ ]:
path = list(data.keys())
datos = pd.read_csv(path[0])
data_y = datos.columns[1]
data_x = datos.columns[0]
nu = np.array(datos[data_x])
L = np.array(datos[data_y])
L_max = max(L)
k = L == L_max
nu_max = nu[k][0]
print(data_y)
print('longitud_onda_max = ', nu_max, 'nm')

## Ajuste datos al modelo de Planck
En esta celda se determinan los párametros del modelo del modelo de Planck:
Enunciarlos.

## Modelo de Planck: 
## $L(\lambda, T) = \frac{2hc^2}{\lambda^5}\frac{1}{e^{\frac{hc}{K_BT\lambda}}-1}$

## Ecuación de ajuste:
## $L(\lambda) = \frac{c_1}{\lambda^5}\frac{1}{e^{\frac{c_2}{\lambda}}-1}$


In [ ]:
def Fun(a, nu):
    #h = 6.6260664e-34
    #c = 2.99792458e8
    #T = 300
    #kb = 1.380648813e-23
    f = np.array([])
    if len(a) == 2:
        c1 = a[0]
        c2 = a[1]
        #f = (2*h*c**2/nu**5)/(np.exp((h*c)/(nu*kb*T))-1)
        f = c1/nu**5/(np.exp(c2/nu)-1)
    else:
        print('Error: la loggitud de (a) debe ser (3) ...')
    return f
def FunInverseModel(xRange=[-np.inf, np.inf], color='b', mark='.', niter=100, label=''):
    """
    FunInverseModel let to calculate the parameters of Fun otained by inverse modellation of current data.
    phiRange: is the range of phi to be used in the modelation
    """
    x, y = nu, L
    dy = 0.1*np.random.ranf(len(x))*y
    y = y + dy
    kx = (x >= xRange[0]) & (x <= xRange[1])
    """
    Parámetros iniciales
    """
    c1 = 100000
    c2 = 10
    p0 = [c1, c2]
    def smr(p, x=x):
        f_fit = Fun(p, x[kx])
        aux = y[kx] - f_fit
        error = np.sqrt(sum(aux**2)/len(aux))
        return error
    search = basinhopping(smr, p0, T=10.0, stepsize=1e-6, niter = niter)
    p = search.x
    x_fit = np.linspace(min(x), max(x), 100)
    # create a new plot
    pa = figure(
        tools="pan,box_zoom,reset,save",
        y_range=[np.min(y), np.max(y)], title='c1 = %0.4f [W*(nm)^4*sr^-1*m^-2];  c2 = %0.4f [um]' %(p[0], p[1]),
        x_axis_label='Long. Onda [um]', y_axis_label='L [W*sr^-1*nm^-1*m^-2]'
    )
    # add some renderers
    pa.circle(x, y, legend_label='Experimental', fill_color="cyan", line_color=None, size=10)
    pa.line(x_fit, Fun(p, x_fit), legend_label="Ajuste", line_width=3, color="black")
    show(pa)
    # show the results
    return search, smr, x

sol, smr, x = FunInverseModel(xRange=[0, np.inf])
c1, c2 = sol.x
print('c1 = ', c1, '[W*(nm)^4*sr^-1*m^-2]') 
print('c2 = ', c2, '[um]')

## Determinación de la constante de Planck y la temperatura del cuerpo.

In [ ]:
#h = 6.6260664e-34
c = 2.99792458e8
kb = 1.380648813e-23
h = (1e-21)*0.5*c1/c**2
print('h = ', h, '[Js]')
T = h*c/(kb*c2*1e-6)
print('T = ', T, '[K]')